# `S3`: Sensor Lab 3: Portable Forceplate

Force plates (or force platforms) are instruments that measure the ground reaction forces generated by a body standing on or moving across them. They are commonly used to quantify balance, gait and other parameters of biomechanics in medicine and sports.

The simplest force plates measure only the vertical component of the force in the geometric center of the platform. More advanced models measure the three-dimensional components of the single equivalent force applied to the surface and its point of application, usually called the centre of pressure (CoP).

In this portable lab you will be working with a very simple CoP forceplate consisting of four [force sensors](https://www.kiwi-electronics.nl/nl/load-sensor-50kg-2813?language=nl-nl&currency=EUR&gclid=Cj0KCQjwxtSSBhDYARIsAEn0thTvnI1bd6WQD_von2DZ8tOxBhw2jOA-A8ubldtaUDM1Rzt_mVxNFhoaAveIEALw_wcB) coupled to [HX711 amplifiers](https://www.sparkfun.com/products/13879) ([further info on load cells](https://learn.sparkfun.com/tutorials/getting-started-with-load-cells)).


## Outline

* [1. Setup Hardware](#Ch1)
  * [1.1 Wire Load Cells to the Connector](#Ch11)
  * [1.2 Seat Load Cell Amps and Raspberry Pi on the Motherboard](#Ch12)
  * [1.3 Wire the Raspberry Pi into the Motherboard](#Ch13)
* [2. Setup Software](#Ch2)
* [3. Control the Force Plate with Python Code](#Ch3)
  * [3.1 Figure Out Which Pin is Connected to Which Corner](#Ch31)
  * [3.2 Calibrate the Force Plate](#Ch32)
  * [3.3 Read Signals from the Load Cells](#Ch33)
* [4. Create an Animated Plot to Visualize the Signals](#Ch4)
* [5. Calculate the Center of Pressure (CoP) on the Force Plate](#Ch5)
  * [5.1 From Force to CoP](#Ch51)
* [6. Save Measurements to a Data File](#Ch6)

## 1. Setup Hardware <a class="anchor" id="Ch1"></a>


### 1.1 Wire Load Cells to the Connector <a class="anchor" id="Ch11"></a>

> ℹ️ **Note**: You may not need to do this part
>
> The PBL organizers may have already wired 4 load cells to a connector for you. If this is the case (i.e. if it looks like all four load cells are already wired into a connector) then you can skip this section.

Each load cell in a force plate has three wires that need to be hooked up the connector. The numbers on the connector are the same but the colour of the wires can be different in your plate compared to the schematic overview. Below the colours are used to indicate the groups belonging to individual loadcells.

<!-- Figure connector and numbers  -->
<div> <img src="images/ConnectorColours.svg" width="600"> </div>

| Number on connector | Pin Function | Belongs to Loadcell |
| :-----------------: | :----------: | :-----------------: |
| 1                   | Exitation +  | 1                   |
| 2                   | Exitation -  | 1                   |
| 3                   | A -          | 1                   |
| 4                   | Exitation +  | 2                   |
| 5                   | Exitation -  | 2                   |
| 6                   | A -          | 2                   |
| 7                   | Not connected| NA                  |
| 8                   | Not connected| NA                  |
| 9                   | Not connected| NA                  |
| 10                  | Exitation +  | 3                   |
| 11                  | Exitation -  | 3                   |
| 12                  | A -          | 3                   |
| 13                  | Exitation +  | 4                   |
| 14                  | Exitation -  | 4                   |
| 15                  | A -          | 4                   |


### 1.2 Seat Load Cell Amps and Raspberry Pi on the Motherboard <a class="anchor" id="Ch12"></a>

The hardware configuration for this lab involves 5 PCBs:

- The Raspberry Pi Zero
- Four HX711 load cell amplifiers
- A motherboard that connects the connector (above) to the load cells

The motherboard is then mounted onto a 3D printed case that is eventually slid underneath the force plate. You have to seat each component on the motherboard. Below are before/after configuration pictures:

<img src="images/SeatComponentsOnPCB.jpg" style="width: 70%;" />
<img src="images/SeatedComponentsOnPCB.jpg" style="width: 70%;" />


### 1.3 Wire the Raspberry Pi into the Motherboard <a class="anchor" id="Ch13"></a>

The motherboard does not electically connect to the Raspberry Pi. This is so that the various pins etc. can be reconfigured. You must connect the Raspberry Pi to the motherboard using jumper cables.

Here is how we recommend you should connect the Raspberry Pi to the motherboard:

| Pin on Motherboard | Pin on Raspberry Pi Zero | Recommended Wire Color |
| :----------------: | :----------------------: | :--------------------: |
| `+3V3`             | `3.3v`                   | Red                    |
| `GND`              | `Ground`                 | Black                  |
| `DU1`              | `GPIO 22`                | (any unique color)     |
| `DU2`              | `GPIO 27`                | (any unique color)     |
| `DU3`              | `GPIO 4`                 | (any unique color)     |
| `DU4`              | `GPIO 14`                | (any unique color)     |
| `Clk`              | `GPIO 17`                | (any unique color)     |

> ℹ️ **Note**: The pin configuration is important
>
> If you use a different pin configuration (e.g. you connect `DU2` to `GPIO 12`), then you should note that down somewhere, because the python code you write later needs to know what is connected to what.

Here's a Pinout diagram for a Raspberry Pi zero:

<img src="images/Raspberry-Pi-Zero-2W-GPIO-Pinout.webp" style="width: 70%;" />

## 2. Setup Software <a class="anchor" id="Ch2"></a>

The lab organizers have already gone through [X0_SoftwareSetup](../X0_SoftwareSetup), which sets up the necessary software for you, before you were given the Pi, so you probably don't need to set up any software.

The configuration script downloads and installs a modified version of `hx711-multi` to `/opt` on your Pi. If you're curious about what it did, you can read through `s3.py`'s source code in the [pbl](../X0_SoftwareSetup/pbl/pbl) module.

> ℹ️ **Problem With Your Pi?**
>
> The course organizers have tried their best to ensure all the configuration options and software you'll need is already installed before the course begins, but we can miss things. If you find that the Pi isn't working for you then you can try:
>
> - Asking for help
> - Running `pbl test s3` in the terminal, which runs some basic checks that ensure things like libraries etc. are installed
> - Reinstalling the necessary software by running `sudo pbl install` in the terminal (⚠️ **warning**: takes a long time)
> - Manually going through the legacy setup guide [here](Legacy/S3_LegacySoftwareSetup.ipynb)

## 3. Control the Force Plate with Python Code <a class="anchor" id="Ch3"></a>


### 3.1 Figure Out Which Pin is Connected to Which Corner <a class="anchor" id="Ch31"></a>

The rest of this lab focuses on taking readings from each load cell and somehow relating them to physical motion on the force plate.

Because there is a chance the connector is wired slightly differently from above, or that jumper cables were connected to the Pi in the wrong order, we will now manually create a pin-to-corner mapping:

- Place the force plate in its measurement position and ensure there is no weight on the force place
- Run `identify.py` in `hx711-multi` (located at `/opt/hx711-multi`) by changing to the `test` directory with:

```bash
cd /opt/hx711-multi/tests
```

- Followed by running `python identify.py Clk DU1 DU2 DU3 DU4`, substituting the pin names with the pin numbers (example: `python3 identify.py 17 22 27 4 14`)
- The `identify.py` script should continuously print raw signals from each pin (e.g. `Pin(22) = 1007` tells you that pin `22` has a "raw" reading value of `1007`)
- Raw readings increase with the amount of force placed on a load cell.
- Start placing weights, or pressing slightly, on each corner of the force plate to identify which pins correspond to which corners of the force plate.
- **Note the pin-to-corner correspondance down in a diagram or table. You will want to know how each pin in python code corresponds to a physical corner of the plate when performing Center of Pressure (CoP) measurements later**


### 3.2 Calibrate the Force Plate <a class="anchor" id="Ch32"></a>

The HX711 Load Cell ADC amp doesn't measure force directly. Instead, it produces a signal (a number) that changes when the force applied to the plate changes. Therefore, an [external calibration](https://www.adamequipment.com/aeblog/internal-calibration-vs-external-calibration) is necessary to map those signals to forces (or weights).

Each HX711 must be calibrated separately. This is because each load cell isn't necessarily identical, which creates variance in raw measurements when compared to the weight that is applied to each load cell.

> ℹ️ **Example**
>
> Placing a one kilogram on the force plate and reading data from one of the HX711s may return a value (signal) of `5000`. This implies that a signal of `5000` maps to `1 kg`. Assuming a linear relationship between the signal and the weight, the weight multiple for that HX711 is therefore `5000`.
>
> Placing the same one kilogram weight on the force plate and reading data from a different HX711 may return a value of (e.g.) `5500`. This implies that a signal of `5500` maps to `1 kg` for the other HX711. Weight multiples are permitted to be different for each load cell due to differences in manufacturing tolerence and assembly.
>
> So, when using four HX711s, you will need to use your calibration (weight multiples) to map four signals (e.g. `5000`, `5500`, `4900`, and `3000`) onto weights/forces (e.g. `1 kg`, `1 kg`, `1 kg`, and `1 kg`).

The `hx711-multi` library comes with a calibration script (`tests/calibrate.py`) that you can use to compute the **calibration values** of **each** HX711 load cell.

For each load cell (e.g. the one attached to `DU1`):

- Remove all weight from the plate
- Change to the `hx711-multi` directory: `cd /opt/hx711-multi`)
- Run `tests/calibrate.py` in a terminal (via `python3 tests/calibrate.py`) or from your editor (e.g. Mu)
- The `calibrate.py` script will prompt you to `Enter SCK/Clock pin`.
- Enter the pin on the Raspberry Pi you attached the `Clk` pin on the motherboard to (default: `17`)
- The `calibrate.py` script will prompt you to `Enter DOut/Measurement pin`.
- Enter the pin on the Raspberry Pi that you attached one of the load cell's data pin on the motherboard (e.g. if `DU1` is attached to GPIO pin `22` on the Pi then you should type `22`)
- When prompted, place a **known** weight on the corner of the plate that corresponds to your load cell
- When prompted, enter the weight, in kilograms, into the program
- The program will then print a calibration value. This is a value you can use to convert a signal from the HX711 to a measurement in Kg. **You should note this value down - you'll be using it in your experiments**.

If you want a more accurate calibration value, you can change the weight to other known values three times. After the third time, just press enter without entering any value as a known weight. The program will then provide you with an average calibration value.

> ⚠️ **Warning**: Remember to note down the calibration value for **each** load cell.
>
> You *need* a calibration value in order to complete the remaining steps. Otherwise, the signals you get from the load cells will not be useful.

### 3.3 Read Signals from the Load Cells <a class="anchor" id="Ch33"></a>

The `/opt/hx711-multi` directory contains example code for reading from multiple HX711 sensors. You can create your own copy of this code and modify it for your needs.

1. Copy `/opt/hx711-multi` to your desktop with `cp -ar /opt/hx711-multi ~/Desktop`

2. Open the example code `~/Desktop/hx711-multi/tests/simple_read_test.py` in our python editor (e.g. `Mu`)

3. At the top of `simple_read_test.py`, change `sck_pin = 17` to match your `Clk` pin assignment (during the hardware setup section)

4. Change `dout_pins` to `dout_pins = [22, 27, 4, 14]` (or whichever GPIO pins you connected the motherboard's `DUx` pins to)

5. In `weight_multiples` enter the four calibration values that you calculated in the previous section. ℹ️ **Note**: the calibration values should be in the same sequence as `dout_pins`

6. To run this script, you can open a terminal, change to the directory, and then use `python` to run it:

```bash
cd ~/Desktop/hx711-multi/tests
python simple_read_test.py
```

7. Run the code and ensure that the output is indeed printed

> ❓**Test Yourself**:
>
> - Where in the `simple_read_test.py` is your output printed?
> - Can you modify the printing part to print something else?

(here is an example of the `simple_read_test.py` content):

In [ ]:
# Example code: simple_read_test.py (should come with the git library)

#!/usr/bin/env python3

from hx711_multi import HX711
from time import perf_counter
import RPi.GPIO as GPIO  # import GPIO

# init GPIO (should be done outside HX711 module in case you are using other GPIO functionality)
GPIO.setmode(GPIO.BCM)  # set GPIO pin mode to BCM numbering

readings_to_average = 10
sck_pin = 1
dout_pins = [2, 3, 4, 14, 15]
weight_multiples = [-5176, -5500, -5690, -5484, -5455]

# create hx711 instance
hx711 = HX711(dout_pins=dout_pins,
              sck_pin=sck_pin,
              channel_A_gain=128,
              channel_select='A',
              all_or_nothing=False,
              log_level='CRITICAL')
# reset ADC, zero it
hx711.reset()
try:
    hx711.zero(readings_to_average=readings_to_average*3)
except Exception as e:
    print(e)
# uncomment below loop to see raw 2's complement and read integers
# for adc in hx711._adcs:
#     print(adc.raw_reads)  # these are the 2's complemented values read bitwise from the hx711
#     print(adc.reads)  # these are the raw values after being converted to signed integers
hx711.set_weight_multiples(weight_multiples=weight_multiples)

# read until keyboard interrupt
try:
    while True:
        start = perf_counter()

        # perform read operation, returns signed integer values as delta from zero()
        # readings aare filtered for bad data and then averaged
        raw_vals = hx711.read_raw(readings_to_average=readings_to_average)

        # request weights using multiples set previously with set_weight_multiples()
        # This function call will not perform a new measurement, it will just use what was acquired during read_raw()
        weights = hx711.get_weight()

        read_duration = perf_counter() - start
        sample_rate = readings_to_average/read_duration
        print('\nread duration: {:.3f} seconds, rate: {:.1f} Hz'.format(read_duration, sample_rate))
        print(
            'raw',
            ['{:.3f}'.format(x) if x is not None else None for x in raw_vals])
        print(' wt',
              ['{:.3f}'.format(x) if x is not None else None for x in weights])
        # uncomment below loop to see raw 2's complement and read integers
        # for adc in hx711._adcs:
        #     print(adc.raw_reads)  # these are the 2's complemented values read bitwise from the hx711
        #     print(adc.reads)  # these are the raw values after being converted to signed integers
except KeyboardInterrupt:
    print('Keyboard interrupt..')
except Exception as e:
    print(e)

# cleanup GPIO
GPIO.cleanup()

## 4. Create an Animated Plot to Visualize the Signals <a class="anchor" id="Ch4"></a>

Since it is hard to check whether the numbers that are printed make sense, we will create an interface that plots the outputted numbers. Let's create a simple interface with which we can see a running data plot while capturing.

> 🏆 **Challenge `S3.4`**: Create an animated plot that visualizes your force plate signals.
>
> It should contain:
>
> - A `matplotlib` plot that updates with running data
> - ⭐⭐ **Optional Challenge #1**: And several lines (e.g. from load cells) are visualized within the figure as (sub)plots
> - ⭐⭐⭐ **Optional Challenge #2**: And there are start/stop buttons to start/stop the animation
>
> 💡 **Tips**:
>
> - This blog helps you with some inital code: https://learn.sparkfun.com/tutorials/graph-sensor-data-with-python-and-matplotlib/speeding-up-the-plot-animation

#### Useful Resources

- [Sparkfun Guide to Plotting Sensor Data](https://learn.sparkfun.com/tutorials/graph-sensor-data-with-python-and-matplotlib/speeding-up-the-plot-animation)
- [Matplotlib API: `FuncAnimation`](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation)
- [Matplotlib API: `add_subplot`](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.add_subplot) (⭐⭐)
- [Matplotlib API: `Button`](https://matplotlib.org/stable/gallery/widgets/buttons.html) (⭐⭐⭐)

In [ ]:
# Create your own code on the Raspberry Pi


## 5. Calculate the Center of Pressure (CoP) on the Force Plate <a class="anchor" id="Ch5"></a>

### 5.1 From Force to CoP <a class="anchor" id="Ch51"></a>

In biomechanics, center of pressure (CoP) is the term given to the point of application of the ground reaction force vector. The ground reaction force vector represents the sum of all forces acting between a physical object and its supporting surface. Analysis of the center of pressure is common in studies on human postural control and gait. Changes in motor control are reflected in changes in the center of pressure.

A forceplate can be used to track the CoP during standing balance. Commercial forceplates (e.g. [Kistler](https://www.kistler.com/en/product/type-9286b/)) have four three-dimensional force sensors in the four corners of the plate. These forceplates are quite expensive (around >10k). In your setup the force sensors are one-dimensional sensors, measuring forces in a single direction only: perpedicular to the forceplate. 

1. Estimate the CoP from the output of the force sensors. Use the following diagram for estimation. 

<div> <img src="images/Kistler_FP_CoP.png" width="600"> </div>
<div> <img src="images/Kistler_FP_CoP_tables.png" width="600"> </div>

2. Implement the CoP estimation in your code. 

__Check yourself: What are the assumptions you had to do to estimate the COP with these sensors?__


In [ ]:
## Write your own code on the RP to extract the CoP from the measurements


## 6. Save Measurements to a Data File <a class="anchor" id="Ch6"></a>

In the previous sections you have extracted and plotted live force plate data using the libraries. However, we cannot analyse data if this isn't saved somewhere.

To do this, you are going to need to know how to generate unique timestamped filenames ([X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb)) and how to write to CSV files ([X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb)). You should then produce a python script, `logging_FP_data.py`, that logs (writes) your force plate data to a file.

> 🏆 **Challenge `S3.6`**: Go through the [X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb) and [X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb) "eXtra Content" materials.
>
> - After going through [X1](../X1_WritingCSVFiles/X1_WritingCSVFiles.ipynb), you should know how to write CSV files
> - After going through [X2](../X2_GeneratingTimestampedFilenames/X2_GeneratingTimestampedFilenames.ipynb), you should know how to generate timestamped file names
> - Adopt (copy) the code you have written up to now to read data from the load cells, convert it into forces, calculate CoPs, plot, etc. into a new Python3 file called `logging_FP_data.py`
> - Write code in `logging_FP_data.py` that does all of the above, but also writes data to a datafile `data/output_yourdatestring.csv` (e.g. `data/output_20220404-114200.csv`)
> - Remember to close your data file at the end of the acquisition
>
> 💡 **Tips**:
>
> - Don't forget to import the `datetime` library at the start of your script
> - Think about how you would like to export the data. What will be your headers? CSV files typically have a header line that describes each column.
> - The rows of your data file will be filled with the force measurements

In [ ]:
# 💡 tip: you may need to build a row of your CSV cell-by-cell

from datetime import datetime

row = []
row.append(datetime.now())  # append a timestamp in the first column
row.append(value1)
row.append(value2)

# (and then write the `row` data array to your data file via a `csv.writer`)